In [52]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np

In [9]:
log_data=pd.read_csv('../Datas/log1.csv',header=None)

In [10]:
log_data

,0,1,2,3
0,busanmovie5,1591524325900,2445509700,...
1,busanmovie1,1591524329458,58600800,...
2,busanmovie6,1591524329683,54576500,...
3,busanmovie1,1591524329894,47667400,...
4,tourguide1,1591524330100,53428500,...
...,...,...,...,...
2828,nationalparkstatus,1591524811658,28219600,...
2829,roadkillstatus,1591524811830,28280000,...
2830,tourguide,1591524811993,49620900,...
2831,endangeredanimals1,1591524812193,25620100,...


In [11]:
log_data.columns=['Dataset','Query Time','Execution Time','ManagerId']

In [12]:
log_data.info

<bound method DataFrame.info of                  Dataset     Query Time  Execution Time ManagerId
0            busanmovie5  1591524325900      2445509700       ...
1            busanmovie1  1591524329458        58600800       ...
2            busanmovie6  1591524329683        54576500       ...
3            busanmovie1  1591524329894        47667400       ...
4             tourguide1  1591524330100        53428500       ...
...                  ...            ...             ...       ...
2828  nationalparkstatus  1591524811658        28219600       ...
2829      roadkillstatus  1591524811830        28280000       ...
2830           tourguide  1591524811993        49620900       ...
2831  endangeredanimals1  1591524812193        25620100       ...
2832          tourguide1  1591524812371        43877400       ...

[2833 rows x 4 columns]>

In [13]:
log_data.describe()

,Query Time,Execution Time
count,2.833000e+03,2.833000e+03
mean,1.591525e+12,3.077180e+07
std,1.377986e+05,4.820137e+07
min,1.591524e+12,1.826380e+07
25%,1.591524e+12,2.528870e+07
50%,1.591525e+12,2.642720e+07
75%,1.591525e+12,2.856920e+07
max,1.591525e+12,2.445510e+09


In [14]:
X=log_data[['Dataset','Query Time','Execution Time']]

In [15]:
X

,Dataset,Query Time,Execution Time
0,busanmovie5,1591524325900,2445509700
1,busanmovie1,1591524329458,58600800
2,busanmovie6,1591524329683,54576500
3,busanmovie1,1591524329894,47667400
4,tourguide1,1591524330100,53428500
...,...,...,...
2828,nationalparkstatus,1591524811658,28219600
2829,roadkillstatus,1591524811830,28280000
2830,tourguide,1591524811993,49620900
2831,endangeredanimals1,1591524812193,25620100


In [16]:
X.describe()

,Query Time,Execution Time
count,2.833000e+03,2.833000e+03
mean,1.591525e+12,3.077180e+07
std,1.377986e+05,4.820137e+07
min,1.591524e+12,1.826380e+07
25%,1.591524e+12,2.528870e+07
50%,1.591525e+12,2.642720e+07
75%,1.591525e+12,2.856920e+07
max,1.591525e+12,2.445510e+09


In [17]:
execute_mean=X[['Execution Time']].mean()
execute_std=X[['Execution Time']].std()
X[['Execution Time']]=(X[['Execution Time']]-execute_mean)/execute_std * 10

C:\Users\jhyun\Anaconda3\envs\MLDL\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [18]:
query_min=X[['Query Time']].min()
X[['Query Time']]=(X[['Query Time']]-query_min)

In [19]:
outliar_removed_index=X[X[['Execution Time']]<2]['Execution Time'].dropna().index
X=X.iloc[outliar_removed_index]

In [20]:
Dataset_Query=X.groupby('Dataset')['Query Time'].sum().to_frame().reset_index()

In [21]:
Dataset_Execution=X.groupby('Dataset')['Execution Time'].mean().to_frame().reset_index()

In [22]:
Dataset_Query_Frequency=X.groupby('Dataset')['Query Time'].count().to_frame().reset_index()
Dataset_Query_Frequency.rename(columns = {'Query Time' : 'Query Count'}, inplace = True)

In [23]:
X=pd.merge(Dataset_Query,Dataset_Execution,on='Dataset')
X=pd.merge(X,Dataset_Query_Frequency,on='Dataset')

In [24]:
count_mean=X[['Query Count']].mean()
count_std=X[['Query Count']].std()
X[['Query Count']]=(X[['Query Count']]-count_mean)/count_std

In [25]:
query_mean=X[['Query Time']].mean()
query_std=X[['Query Time']].std()
X[['Query Time']]=(X[['Query Time']]-query_mean) / query_std

In [26]:
X

,Dataset,Query Time,Execution Time,Query Count
0,busanmovie1,0.674722,-0.857548,0.847707
1,busanmovie2,-1.065483,-0.739944,-0.512810
2,busanmovie3,-1.728874,-0.825591,-1.510523
3,busanmovie4,-1.196107,-0.604405,-1.057018
4,busanmovie5,-0.209938,-0.825927,-0.059305
5,busanmovie6,0.051797,-0.796543,-0.195356
6,endangeredanimals,-0.901212,-1.012544,-1.102368
7,endangeredanimals1,-0.433569,-0.917315,-0.059305
8,koreanhistory,1.408911,-0.286632,1.527966
9,koreanhistory1,-0.368807,-0.177881,-0.512810


In [27]:
query_hot=set(X[X['Query Time']>X['Query Time'].mean()].index)

In [28]:
execution_hot=set(X[X['Execution Time']>X['Execution Time'].mean()].index)

In [29]:
count_hot=set(X[X['Query Count']>X['Query Count'].mean()].index)

In [30]:
print(query_hot)
print(execution_hot)
print(count_hot)

{0, 5, 8, 11, 12, 13, 14, 17, 19, 20, 21, 23, 25}
{1, 3, 8, 9, 10, 22, 23, 24, 25}
{0, 8, 11, 12, 13, 17, 19, 20}


In [31]:
Hot_Dataset_Counts=round(len(X)*0.3)

In [34]:
hot_index=[]

# Query Time and Execution Time and Query Count
intersect=query_hot.intersection(execution_hot).intersection(count_hot)
hot_index=[data for data in intersect]

intersects=[]
#Query Time and Query Count
intersects.extend(query_hot.intersection(count_hot))
#Execution Time and Query Count
intersects.extend(execution_hot.intersection(count_hot))
#Query Time and Execution Time
intersects.extend(query_hot.intersection(execution_hot))
print(intersects)
while len(hot_index) < Hot_Dataset_Counts:
    data=intersects.pop(0)
    if data in hot_index:
        continue
    hot_index.append(data)

[0, 8, 11, 12, 13, 17, 19, 20, 8, 8, 25, 23]


In [35]:
hot_index

[8, 0, 11, 12, 13, 17, 19, 20]

In [36]:
Y=pd.DataFrame(np.zeros(26))

In [37]:
hots=list(hot_index)

In [38]:
Y.loc[hots]=1.0

In [39]:
Y

,0
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,1.0
9,0.0


In [40]:
X=X[["Query Time","Execution Time","Query Count"]]

In [41]:
X

,Query Time,Execution Time,Query Count
0,0.674722,-0.857548,0.847707
1,-1.065483,-0.739944,-0.512810
2,-1.728874,-0.825591,-1.510523
3,-1.196107,-0.604405,-1.057018
4,-0.209938,-0.825927,-0.059305
5,0.051797,-0.796543,-0.195356
6,-0.901212,-1.012544,-1.102368
7,-0.433569,-0.917315,-0.059305
8,1.408911,-0.286632,1.527966
9,-0.368807,-0.177881,-0.512810


In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [47]:
tree=DecisionTreeClassifier()
model=tree.fit(X_train,y_train)

In [48]:
X_test

,Query Time,Execution Time,Query Count
8,1.408911,-0.286632,1.527966
16,-0.057089,-1.031587,-0.104655
0,0.674722,-0.857548,0.847707
24,-1.644317,-0.305742,-1.510523
11,0.287335,-0.821214,0.212799
9,-0.368807,-0.177881,-0.512810
13,2.686636,-1.032205,2.979185
1,-1.065483,-0.739944,-0.512810
23,0.086057,-0.617147,-0.013954


In [54]:
predict=model.predict(X_test)

In [55]:
print("Accuracy: %.2f" % (accuracy_score(y_test, predict) * 100), "%") # 정확도 % 계산

Accuracy: 100.00 %
